In [4]:
# *******Libraries*******
import numpy as np
import matplotlib.pyplot as plt

linear convolution

In [16]:
x=np.array([0,0,0,1,0,0,0,0]) 
m=len(x)
h=np.array([1,2,3,2,0]) 
n=len(h)

h=h.reshape(1,n)
h=np.fliplr(h)
#print(h)

y=np.zeros((m+2*(n-1)) ,dtype=int)
X=np.zeros((m+2*(n-1)) ,dtype=int)
X[n-1:n-1+m] = x

for i in range(len(X)-n):
    y[i]= np.dot(h,X[i:i+n])
    
print("full convolution result:")
print(y)

print('same convoluted result')
same=y[(n-1)//2:(n-1)//2+m]
same

full convolution result:
[0 0 0 1 2 3 2 0 0 0 0 0 0 0 0 0]
same convoluted result


array([0, 1, 2, 3, 2, 0, 0, 0])

In [5]:
# defining the filter/kernel  and the input
w=np.array([[1,2,3],
            [4,5,6],
            [7,8,9]])
f=np.array([[0,0,0,0,0],
            [0,0,0,0,0],
            [0,0,1,0,0],
            [0,0,0,0,0],
            [0,0,0,0,0]])

In [6]:
# Image rotation by 180 degrees

def rot(img):
    img=img.T
    m,n=img.shape
    for i in range(m):
        for j in range(n//2):
            temp=img[i][j]
            img[i][j]=img[i][n-1-j]
            img[i][n-1-j]=temp
    return img

Function for 2-D 'full' and 'same' convolution

In [7]:
def  convn(f,w,conv_type):
    r,c=w.shape
    R,C=f.shape

    #flipping filter f by 180 for convolution
    w=np.flipud(w)
    w=np.fliplr(w)
    #w=rot(w)

    #creating empty output array 
    
    F=np.zeros((R+2*(r-1),C+2*(c-1)),dtype=int)
    out=np.zeros((R+2*(r-1),C+2*(c-1)),dtype=int)
    F[r-1:R+r-1,c-1:C+c-1]=f  #zero padding f  

    
    print("shape of final matrix: "+ str(out.shape))

    for i in range(out.shape[0]-r):
        for j in range(out.shape[1]-c):
            out[i+(r-1)//2][j+(c-1)//2]= np.sum(w * F[i:i+r,j:j+c])
    if conv_type =="full" :       
        print('full 2-D convolution')
        return out    # full parameter 
        
    elif conv_type =="same" :
        print('same 2-D convolution')
        same=out[r-1:R+r-1,c-1:C+c-1]
        return same
    

Function for Valid convolution

In [14]:
def valid_conv(x,h):
    M,N=x.shape
    m,n=h.shape
    
    h=np.flipud(h)
    h=np.fliplr(h)

    
    out=np.zeros((M-m+1,N-n+1))
    print("shape of final matrix" ,out.shape)
    for i in range(out.shape[0]):
        for j in range(out.shape[1]):
#             out[i+((m-1)//2)][j+((n-1)//2)] = x[i:i+m,j:j+n,:]*h
              out[i][j] = np.sum(h*x[i:i+m,j:j+n])
    out=out.astype("uint8")
    print("valid convolution: ")
    print(out)

Running the functions to find results

In [9]:
out=convn(f,w,"full")
print(out)

shape of final matrix: (9, 9)
full 2-D convolution
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 1 2 3 0 0 0]
 [0 0 0 4 5 6 0 0 0]
 [0 0 0 7 8 9 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]


In [10]:
out=convn(f,w,"same")
print(out)

shape of final matrix: (9, 9)
same 2-D convolution
[[0 0 0 0 0]
 [0 1 2 3 0]
 [0 4 5 6 0]
 [0 7 8 9 0]
 [0 0 0 0 0]]


In [15]:
valid_conv(f,w)

shape of final matrix (3, 3)
valid convolution: 
[[1 2 3]
 [4 5 6]
 [7 8 9]]


### 3-D Convolution of image with filter +bias


In [ ]:
def zero_pad(X, pad):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image, 
    as illustrated in Figure 1.
    
    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions
    
    Returns:
    X_pad -- padded image of shape (m, n_H + 2 * pad, n_W + 2 * pad, n_C)
    """
    
    #(≈ 1 line)
    # X_pad = None
    # YOUR CODE STARTS HERE
    #print(X)
    X_pad=np.pad(X,((0,0),(pad,pad),(pad,pad),(0,0)),mode='constant',constant_values=(0,0))
    
    # YOUR CODE ENDS HERE
    
    return X_pad

In [ ]:
def conv_forward(A_prev, W, b, hparameters):
    """
    Implements the forward propagation for a convolution function
    
    Arguments:
    A_prev -- output activations of the previous layer, 
        numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
    b -- Biases, numpy array of shape (1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"
        
    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    """
    
    # Retrieve dimensions from A_prev's shape (≈1 line)  
    # (m, n_H_prev, n_W_prev, n_C_prev) = None
    
    # Retrieve dimensions from W's shape (≈1 line)
    # (f, f, n_C_prev, n_C) = None
    
    # Retrieve information from "hparameters" (≈2 lines)
    # stride = None
    # pad = None
    
    # Compute the dimensions of the CONV output volume using the formula given above. 
    # Hint: use int() to apply the 'floor' operation. (≈2 lines)
    # n_H = None
    # n_W = None
    
    # Initialize the output volume Z with zeros. (≈1 line)
    # Z = None
    
    # Create A_prev_pad by padding A_prev
    # A_prev_pad = None
    
    # for i in range(None):               # loop over the batch of training examples
        # a_prev_pad = None               # Select ith training example's padded activation
        # for h in range(None):           # loop over vertical axis of the output volume
            # Find the vertical start and end of the current "slice" (≈2 lines)
            # vert_start = None
            # vert_end = None
            
            # for w in range(None):       # loop over horizontal axis of the output volume
                # Find the horizontal start and end of the current "slice" (≈2 lines)
                # horiz_start = None
                # horiz_end = None
                
                # for c in range(None):   # loop over channels (= #filters) of the output volume
                                        
                    # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell). (≈1 line)
                    # a_slice_prev = None
                    
                    # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron. (≈3 line)
                    # weights = None
                    # biases = None
                    # Z[i, h, w, c] = None
    # YOUR CODE STARTS HERE
    (m,n_H_prev,n_W_prev,n_C_prev)= A_prev.shape
    (f,f,n_C_prev,n_C)=W.shape
    s=hparameters["stride"]
    p=hparameters["pad"]
    n_H = int((n_H_prev + 2*p - f)/s) +1 
    n_W = int((n_W_prev + 2*p - f)/s) +1
    A=zero_pad(A_prev, p)
    print(A.shape)
    
    Z=np.zeros((m,n_H,n_W,n_C))
    for n in range(m):
        #A= np.zeros((n_H_prev +2*p,n_W_prev +2*p ,n_C_prev))
        
        #print(A.shape)
        #A[p:p+n_H_prev,p:p+n_W_prev,:]=A_prev[n,:,:,:]
        for k in range(n_C):
            for i in range(n_H):
                i_st=i*s ; i_end=i*s+f
                for j in range(n_W):
                    j_st=j*s ;j_end=j*s+f
                    Z[n,i,j,k]=np.sum(A[n,i_st:i_end,j_st:j_end,:]*W[:,:,:,k]) + b[:,:,:,k]               

    print(Z)        

In [ ]:
np.random.seed(1)
A_prev = np.random.randn(2, 5, 7, 4)
W = np.random.randn(3, 3, 4, 8)
b = np.random.randn(1, 1, 1, 8)
hparameters = {"pad" : 1,
               "stride": 2}

Z, cache_conv = conv_forward(A_prev, W, b, hparameters)
z_mean = np.mean(Z)

In [ ]:
def pool_forward(A_prev, hparameters, mode = "max"):
    """
    Implements the forward pass of the pooling layer
    
    Arguments:
    A_prev -- Input data, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    hparameters -- python dictionary containing "f" and "stride"
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    
    Returns:
    A -- output of the pool layer, a numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache used in the backward pass of the pooling layer, contains the input and hparameters 
    """
    
    # Retrieve dimensions from the input shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve hyperparameters from "hparameters"
    f = hparameters["f"]
    stride = hparameters["stride"]
    
    # Define the dimensions of the output
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    
    # Initialize output matrix A
    A = np.zeros((m, n_H, n_W, n_C))              
    print(A.shape)
    # for i in range(None):                         # loop over the training examples
        # for h in range(None):                     # loop on the vertical axis of the output volume
            # Find the vertical start and end of the current "slice" (≈2 lines)
            # vert_start = None
            # vert_end = None
            
            # for w in range(None):                 # loop on the horizontal axis of the output volume
                # Find the vertical start and end of the current "slice" (≈2 lines)
                # horiz_start = None
                # horiz_end = None
                
                # for c in range (None):            # loop over the channels of the output volume
                    
                    # Use the corners to define the current slice on the ith training example of A_prev, channel c. (≈1 line)
                    # a_prev_slice = None
                    
                    # Compute the pooling operation on the slice. 
                    # Use an if statement to differentiate the modes. 
                    # Use np.max and np.mean.
                    # if mode == "max":
                        # A[i, h, w, c] = None
                    # elif mode == "average":
                        # A[i, h, w, c] = None
    
    # YOUR CODE STARTS HERE
    for n in range(m):
        for k in range(n_C):
            for i in range(n_H):
                i_st= i*stride
                i_end= i_st + f
                for j in range(n_W):
                    j_st=j*stride
                    j_end=j_st + f
                    if mode=="max":
                        A[n][i][j][k]= np.max(A_prev[n,i_st:i_end,j_st:j_end,k])
                    elif mode=="average":
                        A[n][i][j][k]= np.mean(A_prev[n,i_st:i_end,j_st:j_end,k])

    